In [2]:
import cv2
import numpy as np

# Extracting the table of marks

In [24]:
def crop_table(image_path):
    image = cv2.imread(image_path)
    gray =cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5,5), 0)
    clahe = cv2.createCLAHE(clipLimit = 2.0, tileGridSize = (8, 8))
    enhanced = clahe.apply(blurred)
    edges = cv2.Canny(enhanced, 50, 150)
    contours, _ =cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    valid_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 1000]
    mask = np.zeros_like(gray)
    cv2.drawContours(mask, valid_contours, -1, (255), thickness=cv2.FILLED)
    result = cv2.bitwise_and(image, image, mask=mask)
    gray =cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    contours, _ =cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    valid_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 1000]
    
    table_coor = []
    for cont in valid_contours:
        x, y, w, h = cv2.boundingRect(cont)
        table_coor.append((x, y, w, h))
        
    return(table_coor)

In [25]:
l=[]
image_path = 'preprocess.jpeg'
tab_coor = crop_table(image_path)
image = cv2.imread(image_path)
for i in tab_coor:
    l.append(i[3])
ind=l.index(max(l))
x, y, w, h = tab_coor[ind]
sp_tab = image[y:y+h, x:x+w]
cv2.imwrite('Cropped.jpg', sp_tab)

True

In [11]:
import cv2
import numpy as np

def crop_table_new(image_path):
    # Read the image
    image = cv2.imread(image_path)
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (9, 9), 0)
    
    # Use adaptive thresholding to create a clear binary image
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV, 11, 2)
    
    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Assume that the table is the largest contour in the bottom half of the image
    possible_tables = [cnt for cnt in contours if cv2.boundingRect(cnt)[1] > gray.shape[0] / 2]
    if not possible_tables:
        raise Exception("No possible tables found in the bottom half of the image")
    
    # Find the largest contour in the possible tables
    table_contour = max(possible_tables, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(table_contour)
    
    # Crop the table from the image
    cropped_table = image[y:y+h, x:x+w]
    
    return cropped_table

# Path to the new image
image_path_new = '11.jpeg' # Replace with your image path

# Crop the table from the new image
cropped_image_new = crop_table_new(image_path_new)

# Save the cropped image to a file
cropped_image_path_new = 'Cropped_new.jpg' # Replace with your desired output path
cv2.imwrite(cropped_image_path_new, cropped_image_new)

# If you want to display the image
# cv2.imshow('Cropped Table', cropped_image_new)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


True

# Crop Cells

In [4]:
image = cv2.imread('Cropped.jpg')
h = image.shape[0] // 5
w = image.shape[1] // 13

for i in range(5): 
    for j in range(13): 
        ystart = i * h
        yend = (i + 1) * h
        xstart = j * w
        xend = (j + 1) * w
        
        if ystart < image.shape[0] and xstart < image.shape[1]:
            box = image[ystart:yend, xstart:xend]
            cv2.imwrite(f'./Cropped_Images/box_{i}_{j}.jpg', box)